In [2]:
import pandas as pd
from parse import PlaceParser
from util import read_place_desc, read_html_from, write_places_to_tsv
from service import PlaceService, SearchEngine

from index import preprocess, ConjunctiveIndex

from os import listdir

from os.path import isfile, join
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import os
import time
from sklearn.feature_extraction.text import TfidfVectorizer


# Data collection
##  1.1. Get the list of places





In [3]:
with open('most_popular_places.txt', 'w') as file:
    for i in tqdm(range(300)):
        
        url = 'https://www.atlasobscura.com/places?page='+str(i+1)+'&sort=likes_count'
        list_page = requests.get(url)
        list_soup = BeautifulSoup(list_page.text)
        list_places = [x['href'] for x in list_soup.find_all('a', {'class':"content-card content-card-place"})]
        for place in tqdm(list_places):
            file.write('https://www.atlasobscura.com' + str(place))
            file.write('\n')

  1%|▎                                          | 2/300 [00:08<21:48,  4.39s/it]


KeyboardInterrupt: 

## 1.2. Crawl places

In [4]:
f= open('most_popular_places.txt', 'r')
for j, url in enumerate(f):
    
    if j %18 == 0:
        dir_path = f'page{j//18+1}'
        if not os.path.isdir(dir_path):
            os.mkdir(dir_path)
            
    response = requests.get(url.strip())
    with open(f'{dir_path}/{j+1}.html','w') as file:
        print()
        file.write(response.text)
        
    time.sleep(1)

# Parse Pages

In [5]:
htmls = [read_html_from(f'./pages/{file}') for file in listdir('./pages') if isfile(join('./pages', file))]
   
places = [PlaceParser(html).parse() for html in htmls]

tsv_path = write_place_to_tsv('./', places)

names, descriptions = read_place_desc(tsv_path)


NameError: name 'write_place_to_tsv' is not defined

# Create Index

In [ ]:
index = ConjunctiveIndex()
for name, desc in zip(names.values, descriptions.values):
    desc = preprocess(desc)

    for token in desc:
        index.put(token, name)

# Create Search Engine

In [ ]:
place_service = PlaceService()
place_service.load('./places.tsv')

search_engine = SearchEngine(index, place_service)

# Run Search

In [ ]:
search_engine.query('the Clock Tower Building')

In [ ]:
webdatad = descriptions.values
webdatat = names.values


In [ ]:
dic={}
for x,y in zip(webdatad,webdatat):
    dic[y] = preprocess(x,unique = False)

In [ ]:
tfidf = TfidfVectorizer(input='content', lowercase=False, tokenizer=lambda text:text)

result = tfidf.fit_transform(dic.values())

result_dense = result.todense()

result_dense_list = result_dense.tolist()



In [ ]:
query = preprocess(' new would  ' )
try :
    print(p[query].loc[~(p[query] == 0).any(axis=1)])

except KeyError:
    print(f"No documents cointans{query}")



In [ ]:
p[query].loc[~(p[query] == 0).any(axis=1)]